Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

При выполнении данной работы будут выполняться следующие этапы исследования:
1. Общий анализ данных.
2. Предобработка данных.
3. Исследование данных.
4. Общий вывод.

**1. Общий анализ данных**

In [34]:
import pandas as pd

In [35]:
data = pd.read_csv('/datasets/data.csv')

In [36]:
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**2. Предобработка данных**

In [38]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаем на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [39]:
median_data = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data ['total_income'].fillna(median_data)
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода abs()

In [40]:
data['days_employed'] = data['days_employed'].abs()
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях.

In [41]:
median_employed = data.groupby('income_type')['days_employed'].median()
print(median_employed)

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, этот столбец не понадобится для исследования, оставляем без изменений.

In [42]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


В столбце children есть два аномальных значения. Удаляем строки, в которых встречаются такие аномальные значения из датафрейма data.

In [43]:
data = data[data['children']!=-1]
data = data[data['children']!=20]
print(data)

       children  days_employed  dob_years education  education_id  \
0             1    8437.673028         42    высшее             0   
1             1    4024.803754         36   среднее             1   
2             0    5623.422610         33   Среднее             1   
3             3    4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1    4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1    2113.346888         38   среднее             1   
21523         3    3112.481705         38   среднее             1   
21524         2    1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

In [44]:
print(data['children'].unique())

[1 0 3 2 4 5]


Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [45]:
median_employed = data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'] = data ['days_employed'].fillna(median_employed)

In [46]:
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [47]:
data['total_income'] = data['total_income'].astype('int')

In [48]:
print(data.duplicated().sum())

54


In [49]:
data = data.drop_duplicates()

In [50]:
data['education'] = data['education'].str.lower()

На основании диапазонов, указанных ниже, создадим в датафрейме data столбец total_income_category с категориями:

0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'. Используйте собственную функцию с именем categorize_income() и метод apply().

In [51]:
def categorize_income(sum_inc):
    if sum_inc<=30000:
        return 'E'
    if sum_inc<=50000:
        return 'D'
    if sum_inc<=200000:
        return 'C'
    if sum_inc<=1000000:
        return 'B'
    if sum_inc>=1000001:
        return 'A'

In [52]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
print(data)

       children  days_employed  dob_years education  education_id  \
0             1    8437.673028         42    высшее             0   
1             1    4024.803754         36   среднее             1   
2             0    5623.422610         33   среднее             1   
3             3    4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1    4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1    2113.346888         38   среднее             1   
21523         3    3112.481705         38   среднее             1   
21524         2    1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

In [53]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

Используем собственную функцию с именем categorize_purpose() и метод apply(). Изучим данные в столбце purpose и определим, какие подстроки помогут правильно определить категорию.

In [54]:
def categorize_purpose(category):
    if 'жиль' in category:
        return 'операции с недвижимостью'
    if 'недвижим' in category:
        return 'операции с недвижимостью'
    if 'образов' in category:
        return 'получение образования'
    if 'свадь' in category:
        return 'проведение свадьбы'
    if 'авто' in category:
        return 'операции с автомобилем'

In [55]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [56]:
print(data)

       children  days_employed  dob_years education  education_id  \
0             1    8437.673028         42    высшее             0   
1             1    4024.803754         36   среднее             1   
2             0    5623.422610         33   среднее             1   
3             3    4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1    4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1    2113.346888         38   среднее             1   
21523         3    3112.481705         38   среднее             1   
21524         2    1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

**3. Исследование данных**

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [57]:
children_group = data['children'].value_counts()
print(children_group)

0    14107
1     4809
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


В столбце "debt" количество невозвращенных в срок кредитов. Сгруппируем данные по количеству детей, и подсчитаем, сколько невовремя возвращенных кредитов было в каждой группе.

In [58]:
children_debt=data.groupby('children')['debt'].sum()
print(children_debt)

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64


Теперь можно вычислить конверсию: сколько невовремя возвращенных кредитов было у людей с разным количеством детей по отношению к общему количеству кредитов для данной группы клиентов

In [59]:
conversion = children_debt/children_group
print(conversion)

children
0    0.075353
1    0.092327
2    0.094542
3    0.081818
4    0.097561
5    0.000000
dtype: float64


Вывод: Прямой зависимости между количеством детей и возвратом кредита в срок нет. Однако очевидно, что клиенты без детей возвращают кредит в срок чаще. Конверсии по группам заемщиков с тремя и более детьми не являются показательными, поскольку величина выборки для данных групп лиц очень мала.

 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [60]:
status_group = data['family_status'].value_counts()
print(status_group)

женат / замужем          12266
гражданский брак          4146
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64


Сгруппируем данные по семейному статусу, и посчитаем, сколько кредитов с нарушением сроков возврата было в каждой группе

In [61]:
status_debt=data.groupby('family_status')['debt'].sum()
print(status_debt)

family_status
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
гражданский брак         385
женат / замужем          927
Name: debt, dtype: int64


Рассчитаем конверсию: относительную величину кредитов с нарушением сроков возврата к общему количеству кредитов для групп лиц с разным семейным положением

In [62]:
conversion = status_debt/status_group
print(conversion)

Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.092861
женат / замужем          0.075575
dtype: float64


Вывод: Женатые/замужние или ранее состоявшие в браке люди являются более дисциплинированными в возврате кредитов в срок по сравнению с людьми, не состоящими в браке. Самыми дисциплинированными по возвратам кредитов являются вдовцы(вдовы). Примерно одинаковой по величине является группа людей с семейным положением - "в разводе". Вероятность возврата кредита для данной группы лиц примерно так же высока. Чуть хуже по степени благонадежности являются клиенты, состоящие в браке. Однако, с учетом величины группы данных лиц по сравнению с остальными группами, вероятность возврата кредита является очень высокой. Значительно отстают по степени надежности заемщики, находящиеся в гражданском браке. Наибольшая вероятность нарушения сроков возврата кредита присуща группе лиц, не состоящим в браке.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Находим количество кредитов для групп лиц с разным уровнем дохода

In [63]:
categorize_group = data['total_income_category'].value_counts()
print(categorize_group)

C    15938
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64


In [64]:
categorize_debt=data.groupby('total_income_category')['debt'].sum()
print(categorize_debt)

total_income_category
A       2
B     354
C    1353
D      21
E       2
Name: debt, dtype: int64


In [65]:
categorize_debt=data.groupby('total_income_category')['debt'].sum()
conversion = categorize_debt/categorize_group
print(conversion)

A    0.080000
B    0.070602
C    0.084891
D    0.060172
E    0.090909
dtype: float64


Вывод: Прямой зависимости между увеличением дохода и количеством нарушений сроков возврата кредита нет. Конверсия для групп "А" и "Е" не является показательной, т.к. группа лиц с такими величинами доходов в данной таблице очень мал. Лучшими по надежности в возвратах кредитов являются лица, с величиной доходов категории "D". Менее дисциплинированными являются клиенты с доходами категории "В". Вероятность нарушения сроков возврата кредита наиболее высока для лиц, с величиной доходов категории "С". Данная группа является самой многочисленной среди всех клиентов.

Как разные цели кредита влияют на его возврат в срок?

In [66]:
purpose_group = data['purpose_category'].value_counts()
print(purpose_group)

операции с недвижимостью    10754
операции с автомобилем       4281
получение образования        3989
проведение свадьбы           2324
Name: purpose_category, dtype: int64


In [67]:
purpose_debt=data.groupby('purpose_category')['debt'].sum()
print(purpose_debt)

purpose_category
операции с автомобилем      400
операции с недвижимостью    780
получение образования       369
проведение свадьбы          183
Name: debt, dtype: int64


Рассчитаем конверсию для каждой цели кредита

In [69]:
conversion = purpose_debt/purpose_group
print(conversion)

операции с автомобилем      0.093436
операции с недвижимостью    0.072531
получение образования       0.092504
проведение свадьбы          0.078744
dtype: float64


Вывод: Самыми благонадежными явлются займы для операций с недвижимостью. Также именно они являются самыми популярными среди клиентов банка, что говорит о высокой степени вероятности возврата кредита в срок. Чуть больше вероятность нарушения условий кредита у лиц, обращающихся за средствами для проведения свадьбы. Вероятность нарушения сроков возврата кредита значительно выше у кредитов для получения образования. Самыми ненадежными являются заемщики, получающие кредит для операций с автомобилем.

Приведите возможные причины появления пропусков в исходных данных.

Появление пропусков возможно из-за ошибок при внесении данных или из-за умышленного их не заполнения(пользователь не захотел оставлять информацию)

Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Медианное значение - середина между всеми величинами, т.е. значение не выделяющееся в общем ряду числовых значений представленных данных. Медиана не окажет значительного влияния при проведении статистического анализа (например, вычислении среднего арифметического) или аналитических исследований. Медиана выполнит главную задачу, которая стоит при заполнении пропусков - появятся данные, которые позволят провести верные вычисления.

**4.  Общий вывод.**

На основании предоставленной статистики о платёжеспособности клиентов проведено исследование влияния различных факторов на своевременный возврат кредита заемщиком.

Для построения модели скоринга - системы оценки способности потенциального заемщика вернуть кредит в срок, были проверены следующие гипотезы:

Есть ли зависимость между количеством детей и возвратом кредита в срок?
Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?
Анализ данных предоставленной статистики дал следующие результаты.

1. Гипотеза 1 частично подтвердилась.

Более благонадежными являются клиенты банка, у которых нет детей. Прослеживается возрастание доли нарушений возврата кредитов по мере увеличения количества детей для клиентов с 1 и 2 детьми.

0.075129 – доля нарушений возврата кредитов у заемщиков без детей.

0,092154 - доля нарушений возврата кредитов у заемщиков с 1 ребенком.

0,094404 - доля нарушений возврата кредитов у заемщиков с 2 детьми.

Однако недостаточно данных для исследования зависимости у клиентов с 3 и более детьми.

Рекомендация: провести дополнительное исследование на большей выборке данных.

2. Гипотеза 2 подтверждена.

Люди, состоявшие в браке, но теперь являющиеся вдовцами(вдовами) или разведенными или состоящие в браке в текущий момент - вернут кредит с большей вероятностью. Клиенты, не состоявшие в браке или состоящие в гражданском браке являются значительно менее надежными заемщиками.

Рекомендация: разница в величине долей между двумя данными категориями всего 0,02%, необходимо присваивать соответствующий вес для данного критерия в системе оценки потенциального заемщика.

3. Гипотеза 3 не подтвержена.

Прямой зависимости между увеличением дохода и количеством нарушений сроков возврата кредита нет. Конверсия для групп "А" и "Е" не является показательной, т.к. группа лиц с такими величинами доходов в данной таблице очень мал.

Рекомендация: провести дополнительное исследование на большей выборке данных.

4. Гипотеза 4 подтверждена.

Самыми благонадежными являются займы для операций с недвижимостью. Также именно они являются самыми популярными среди клиентов банка, что говорит о высокой степени вероятности возврата кредита в срок. Чуть больше вероятность нарушения условий кредита у лиц, обращающихся за средствами для проведения свадьбы. Вероятность нарушения сроков возврата кредита значительно выше у кредитов для получения образования. Самыми ненадежными являются заемщики, получающие кредит для операций с автомобилем.

0,072140 – доля нарушений сроков возврата кредита для операций с недвижимостью

0,079216 - доля нарушений сроков возврата кредита для проведения свадьбы

0,091994 - доля нарушений сроков возврата кредита для получения образования

0,093395 - доля нарушений сроков возврата кредита для операций с автомобилем

Рекомендация: разница в величине долей между двумя данными категориями всего 0,02%, необходимо присваивать соответствующий вес для данного критерия в системе оценки потенциального заемщика.

Общие рекомендации:

Для построения модели скоринга рекомендуется использовать данные о зависимости между семейным положением и зависимости между целями кредита и возвратом кредита в срок, с учетом соответствующего веса для данных критериев в системе оценки потенциального заемщика.

Необходимо провести дополнительное исследование на большей выборке данных для гипотез о зависимости между количеством детей, уровнем дохода и вероятностью нарушения условий кредита.